In [98]:
import numpy as np
import pandas as pd
from py2neo import Graph, Node, Relationship

import requests
from bs4 import BeautifulSoup
import os

from tqdm import tqdm, trange

In [120]:
class playsFor(Relationship): pass
class won(Relationship): pass
class wentTo(Relationship): pass

In [111]:
url = 'https://www.basketball-reference.com/leagues/NBA_2022_per_game.html'
page = requests.get(url)
page.content[13200:13300]


mvps = pd.read_csv("MVPlist.csv", sep=';', )

In [112]:
soup = BeautifulSoup(page.content, 'html.parser')

table = soup.find_all(class_="full_table")
head = soup.find_all(class_="thead")

In [113]:
col_names = [_.text for _ in head][0]
col_names = col_names.split('\n')[2:-1]

In [121]:
players = []
for i in range(len(table)):
    player_ = []

    for td in table[i].find_all("td"):
        player_.append(td.text)

    players.append(player_)

df = pd.DataFrame(players, columns=col_names)


In [109]:
df[df["Player"]=='Thaddeus Young']

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
600,Thaddeus Young,PF,33,TOT,52,1,16.3,2.7,5.2,.518,...,.469,1.5,2.5,4.0,2.0,1.0,0.3,1.0,1.6,6.2


In [119]:
df2 = pd.read_csv("Players-college.csv").dropna()
df2

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
6,Jordan Mickey,Boston Celtics,55.0,PF,21.0,6-8,235.0,LSU,1170960.0
7,Kelly Olynyk,Boston Celtics,41.0,C,25.0,7-0,238.0,Gonzaga,2165160.0
...,...,...,...,...,...,...,...,...,...
449,Rodney Hood,Utah Jazz,5.0,SG,23.0,6-8,206.0,Duke,1348440.0
451,Chris Johnson,Utah Jazz,23.0,SF,26.0,6-6,206.0,Dayton,981348.0
452,Trey Lyles,Utah Jazz,41.0,PF,20.0,6-10,234.0,Kentucky,2239800.0
453,Shelvin Mack,Utah Jazz,8.0,PG,26.0,6-3,203.0,Butler,2433333.0


In [122]:
driver = Graph("bolt://localhost:7687", auth=('neo4j', '1234'))
driver.delete_all()

In [130]:
# create nodes

def createPlayerNode(teamNode, college, mvp, **kwargs):
    n = Node("PLAYER", **kwargs)
    n.__primarykey__='NAME'
    n.__primarylabel__='PLAYER'
    driver.merge(n)

    col = wentTo(n, college)
    driver.merge(col)

    rel = playsFor(n, teamNode)
    driver.merge(rel)

    if n["NAME"] in mvps["player"].values:
        rel_w = won(n, mvp)
        driver.merge(rel_w)
    return n

def createTeamNode(**kwargs):
    n = Node("TEAM", **kwargs)
    n.__primarykey__='NAME'
    n.__primarylabel__='TEAM'
    driver.merge(n)

    return n

def createCollegeNode(**kwargs):
    n = Node("COLLEGE", **kwargs)
    n.__primarykey__='NAME'
    n.__primarylabel__='COLLEGE'
    driver.merge(n)

    return n

def createMVPnode():
    n = Node("PRIZE", NAME='MVP')
    n.__primarykey__='NAME'
    n.__primarylabel__='PRIZE'
    
    driver.merge(n)
    return n

In [131]:
mvpnode = createMVPnode()

for i in trange(len(df2)):
    row = df2.iloc[i,:]

   # player_args = {
   #     "NAME": row["Player"],
   #     "AGE": row["Age"],
   #     "GAMES": row["G"],
   #     "GAMES_STARTED": row["GS"],
   #     "MINUTES_PLAYED": row["MP"],
   #     "POINTS":row["PTS"],
   #     "REBOUNDS":row["TRB"],
   #     "ASSISTS":row["AST"]
   # }
#
   # team_args = {
   #     "NAME": row["Tm"]
   # }
    print(row["College"])

    college_args = {
        "NAME": row["College"]
    }

    team_args = {
        "NAME": row["Team"]
    }

    player_args = {
        "NAME":     row["Name"],
        "NUMBER":   row["Number"],
        "POSITION": row["Position"],
        "AGE":      row["Age"],
        "HEIGHT":   row["Height"],
        "WEIGHT":   row["Weight"],
        "SALARY":   row["Salary"]
    }

    c = createCollegeNode(**college_args)
    t = createTeamNode(**team_args)
    p = createPlayerNode(t,c, mvpnode, **player_args)

  0%|          | 0/364 [00:00<?, ?it/s]

Texas


  0%|          | 0/364 [00:00<?, ?it/s]


TypeError: Values of type <class 'numpy.float64'> are not supported

In [108]:
mvps["player"].unique()

array(['Nikola Jokic', 'Giannis Antetokounmpo', 'James Harden',
       'Russell Westbrook', 'Stephen Curry', 'Kevin Durant',
       'LeBron James', 'Derrick Rose'], dtype=object)